### Interface Conductivity


       P            e                E
     --o------------|----------------o--

       |-- (dx)e- --|---- (dx)e+ ----|
       |------------ dx -------------|

For a steady, no-source, one-dimensional situation in which coductivity varies in a stepwise fashion from on e control volume to the next, the interface-conductivity formulat is given as



$$

k_e = \left(\frac{1-f_e}{k_P} + \frac{f_e}{k_E} \right) ^{-1}

\\
{

  f_e = \frac{(\delta x)_{e+}}{(\delta x)_{e}}
}

$$




In [ ]:
def interface_k(k_P, k_E, dx, dxe):
    f_e = dxe/dx
    return ((1-f_e)/k_P + f_e/k_E)**-1

In [52]:
import numpy as np
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve


node_type_def = np.dtype([
    ('w', '<f4'),
    ('temp', '<f4')
])

T_A = 100
T_B = 500
T_init = 0

width = 0.5
n_nodes = 5

dx = width/n_nodes

k_e = 1000
A = A_e = A_w = 1*10e-3

nodes_list = [
    (0, T_A), # left boundary
    (dx, T_init),
    (dx, T_init),
    (dx, T_init),
    (dx, T_init),
    (dx, T_init),
    (0, T_B) # right boundary
]

nodes = np.array(nodes_list, dtype=node_type_def)

# This gives the distance betweeen the centers of adjacent control volumes
# dx_we = nodes[1:-1]['w']*0.5 + nodes[0:-1]['w']*0.5

dx_we = nodes[1:]['w']*0.5 + nodes[0:-1]['w']*0.5
# dx_e = nodes[1:-1]['w']*0.5 + nodes[2:]['w']*0.5

a_ws = k_e * A_w / dx_we
a_es = k_e * A_e / dx_we



a_ps = a_ws[1:] + a_es[:-1]

a_ps = np.pad(a_ps, (1, 1), mode='constant', constant_values=(0, 0))

# a_ws = np.pad(a_ws, (1, 0), mode='constant')

NORMALISER = 1e30

a_ps[0] = a_ps[-1] = 1*NORMALISER

mat = [-a_ws, a_ps, -a_es]

b = np.zeros((7,1))

b[0] = 100*NORMALISER
b[-1] = 500*NORMALISER


A = diags(mat, [-1, 0, 1], format='csr')

x = spsolve(A, b)

print(A.toarray())
print(x)


[[ 1.e+30 -2.e+02  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00]
 [-2.e+02  3.e+02 -1.e+02  0.e+00  0.e+00  0.e+00  0.e+00]
 [ 0.e+00 -1.e+02  2.e+02 -1.e+02  0.e+00  0.e+00  0.e+00]
 [ 0.e+00  0.e+00 -1.e+02  2.e+02 -1.e+02  0.e+00  0.e+00]
 [ 0.e+00  0.e+00  0.e+00 -1.e+02  2.e+02 -1.e+02  0.e+00]
 [ 0.e+00  0.e+00  0.e+00  0.e+00 -1.e+02  3.e+02 -2.e+02]
 [ 0.e+00  0.e+00  0.e+00  0.e+00  0.e+00 -2.e+02  1.e+30]]
[ 99.9999985  139.99999789 219.99999669 299.99999549 379.99999428
 459.99999308 499.99999248]
